In [3]:
import requests, time, math, threading
import sqlite3, os
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from queue import Queue
from selenium_tools import get_chrome, find_element, get_soup
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

In [4]:
url = "https://www.google.com/maps/"
datas = pd.read_csv("Taipei_to_hots_copy1.csv", encoding = "utf-8-sig")
datas

,Unnamed: 0.1,Unnamed: 0,ID,shopName,shopAddress,shopPhone,shopHours,shopPhoto
0,0,0,0,紫明溪步道,台北市北投區中興路12號附近（陽明書屋附近）,02-2759-3001,24小時開放,093f0a5d.jpg \n 71a97dc8.jpg \n bd396b43.jpg \...
1,1,1,1,冷水坑公共溫泉浴室,台北市士林區菁山路101巷175號,02-2861-0036,週二至日06：00～08：30、10：00～12：30、14：00～16：30、18：00～...,9cc7ae53.jpg \n a6fd224c.jpg \n 8111757c.jpg \...
2,2,2,2,中國信託文薈館CTBC Museum,台北市南港區經貿二路168號1樓（中國信託金融園區A棟1樓）,02-3327-7777#1099,週二至日11：00～17：30（週一休）,659c98e5.jpg \n 83e1b38d.jpg \n 24c0f290.jpg \...
3,3,3,3,大湖公園,台北市內湖區成功路五段31號,02-3327-7777#1099,24小時開放,e608fa4e.jpg \n 8ebb6ca2.jpg \n 13e813b9.jpg \...
4,4,4,4,和興炭坑,台北市信義區信義路五段150巷471弄,02-2759-3001,週一至日09：00～17：00,0825053e.jpg \n edf45ca7.jpg \n 15f09961.jpg \...
...,...,...,...,...,...,...,...,...
150,150,150,150,璐露野生活Luluyelife,台北市大安區光復南路420巷20號1樓,02-2707-5252,週一至日09：00～18：00,b0342549.jpg \n f7d4060e.jpg \n c5e5898f.jpg \...
151,151,151,151,花博公園（新生園區）,台北市中山區新生北路三段105號,02-2585-0192,24小時開放,3b87489a.jpg \n 7928dcfd.jpg \n e370ade7.jpg
152,152,152,152,劍潭山親山步道,台北市中山區中山北路四段71巷2弄69號,02-2759-3001,24小時開放,e5fb701a.jpg \n 2c1ce6b5.jpg \n 60332644.jpg \...
153,153,153,153,臺北市兒童新樂園,台北市士林區承德路五段55號,02-2181-2345,週二至五09：00～17：00／週六、日09：00～18：00（週一休，非寒暑假）,ad1a9807.jpg \n 94168796.jpg \n ab2ccfe7.jpg \...


In [5]:
def for_comments(name):
    while True:
        chrome = get_chrome(url, hide = True)
        if "yZqPAf" in str(chrome.page_source):
            chrome = get_chrome(url, hide = True)
        else:
            break
    chrome.refresh()
    #網頁最大化
    chrome.maximize_window()
    xpath1 = "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input"
    element = find_element(chrome, xpath = xpath1)
    element.clear()
    element.click()
    element.send_keys(name + "\n")
    time.sleep(2)
    soup = BeautifulSoup(chrome.page_source, "html.parser")
    if "RZ66Rb" not in str(chrome.page_source):
        url_new = soup.find("a", class_ = "hfpxzc").get("href")            
        chrome.quit()
        chrome = get_chrome(url_new, hide = False)
        time.sleep(0.5)
        #網頁最大化
        chrome.maximize_window()
        chrome.refresh()    
    elif "RZ66Rb" in str(chrome.page_source):
        time.sleep(0.5)
    soup = BeautifulSoup(chrome.page_source, "html.parser") 
    com_list = [name]
    if "Gpq6kf fontTitleSmall" in str(chrome.page_source):
        try:
            try:
                xpath = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"
                element = chrome.find_element(By.XPATH, xpath)
                element.click()                
            except Exception as e:
                xpath = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[3]"
                element = chrome.find_element(By.XPATH, xpath)
                element.click()            
        except Exception as e:
            try:
                xpath = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"
                element = chrome.find_element(By.XPATH, xpath)
                element.click()                
            except Exception as e:
                xpath = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[3]"
                element = chrome.find_element(By.XPATH, xpath)
                element.click()      
        time.sleep(0.5)
        try:
            try: 
                comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)                     
        except Exception as e:
            try: 
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)   
        time.sleep(0.5)
        try:
            try: 
                comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)                     
        except Exception as e:
            try: 
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)  
        time.sleep(0.5)
        soup2 = BeautifulSoup(chrome.page_source, "html.parser")
        comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
        time.sleep(2)
        for i in comments:
            if "w8nwRe" in str(i): 
                element = chrome.find_element(By.CLASS_NAME, 'w8nwRe')
                element.click()
        time.sleep(0.5)
        try:
            try: 
                comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)                     
        except Exception as e:
            try: 
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
        time.sleep(0.5)
        soup2 = BeautifulSoup(chrome.page_source, "html.parser")
        comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
        chrome.quit()
        print(name + "SuccessFul!")
        for com in comments:
            if "wiI7pd" in str(com):
                com_list.extend(["姓名: " + com.find(class_ = "d4r55").text, "評論: " + com.find("span", class_ = "wiI7pd").text])
            else:
                com_list.extend(["姓名: " + com.find(class_ = "d4r55").text, "評論: " + "很讚"])
    else:
        com_list = [name, "暫無評價"]      
    return com_list

In [8]:
try:
    while True:
        chrome = get_chrome(url, hide = False)
        if "yZqPAf" in str(chrome.page_source):
            chrome = get_chrome(url, hide = False)
        else:
            break
    chrome.refresh()
    #網頁最大化
    chrome.maximize_window()
    xpath1 = "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input"
    element = find_element(chrome, xpath = xpath1)
    element.clear()
    element.click()
    element.send_keys(datas["shopName"][149] + "\n")
except Exception as e:
    print(e)

In [15]:
chrome.quit()

#### 有選項

In [9]:
try:
    if "Nv2PK Q2HXcd THOPZb" in str(chrome.page_source):
        soup = BeautifulSoup(chrome.page_source, "html.parser")
        num = -1
        for number in soup.find_all("div", class_ = "Nv2PK Q2HXcd THOPZb"):
            number_text = eval(number.find("span", class_ = "UY7F9").text)
            if number_text > num:
                num = number_text
                url_new = number.find("a", class_ = "hfpxzc").get("href")
        chrome.quit()
        chrome = get_chrome(url_new, hide = False)
        time.sleep(0.5)
        #網頁最大化
        chrome.maximize_window()
        chrome.refresh()    
    elif "RZ66Rb" in str(chrome.page_source):
        time.sleep(0.5)
    try:
        xpath = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"
        element = chrome.find_element(By.XPATH, xpath)
        element.click()
    except Exception as e:       
        xpath = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"
        element = chrome.find_element(By.XPATH, xpath)
        element.click()
    time.sleep(0.5)
    try:        
        comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
    except Exception as e:
        comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)   
    time.sleep(0.5)
    try:        
        comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane1 = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = 0", pane1)
    except Exception as e:
        comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane1 = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = 0", pane1) 
    time.sleep(0.5)
    soup2 = BeautifulSoup(chrome.page_source, "html.parser")
    comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
    time.sleep(2)
    for i in comments:
        if "w8nwRe" in str(i): 
            element = chrome.find_element(By.CLASS_NAME, 'w8nwRe')
            element.click()
    time.sleep(0.5)
    try:        
        comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane1 = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = 0", pane1)
    except Exception as e:
        comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane1 = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = 0", pane1) 
    time.sleep(0.5)
    
    soup2 = BeautifulSoup(chrome.page_source, "html.parser")
    comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
    print(len(comments))
    chrome.quit()
#     print(comments)
    com_list = [name]
    for com in comments:
        if "wiI7pd" in str(com):
            com_list.extend(["姓名: " + com.find(class_ = "d4r55").text, "評論: " + com.find("span", class_ = "wiI7pd").text])
        else:
            com_list.extend(["姓名: " + com.find(class_ = "d4r55").text, "評論: " + "很讚"])
    print(pd.DataFrame(com_list))    
except Exception as e:
    print(e)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"}
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF6140C5E42+3538674]
	(No symbol) [0x00007FF613CE4C02]
	(No symbol) [0x00007FF613B95AEB]
	(No symbol) [0x00007FF613BDBF4E]
	(No symbol) [0x00007FF613BDC0CC]
	(No symbol) [0x00007FF613C1E477]
	(No symbol) [0x00007FF613BFF0EF]
	(No symbol) [0x00007FF613C1BDE3]
	(No symbol) [0x00007FF613BFEE53]
	(No symbol) [0x00007FF613BCF514]
	(No symbol) [0x00007FF613BD0631]
	GetHandleVerifier [0x00007FF6140F6CAD+3738973]
	GetHandleVerifier [0x00007FF61414C506+4089270]
	GetHandleVerifier [0x00007FF614144823+4057299]
	GetHandleVerifier [0x00007FF613E15C49+720121]
	(No symbol) [0x00007FF613CF126F]
	(No symbol) [0x00007FF613CEC304]
	(No symbol) [0x00007FF613CEC432]
	(No symbol) [0x00007FF613CDBD04]
	BaseThreadInitThunk [0x00007FFC9241257D+29]


In [15]:
try:
    all_comments = []
    chrome = None
    for name in datas["shopName"][148:]:        
        while True:
            try:
                com_list = for_comments(name)
                if com_list[2:] == []:
                    com_list = for_comments(name)
            except Exception as e:
                if chrome != None:
                    chrome.quit()
                com_list = for_comments(name)
                if com_list[2:] == []:
                    com_list = for_comments(name)
            else:
                break
        all_comments.append(com_list)
        time.sleep(2)
    print(all_comments)          
except Exception as e:
    print(e)

HA憤怒屋SuccessFul!
HA憤怒屋SuccessFul!
'NoneType' object has no attribute 'text'


In [12]:
df1 = pd.DataFrame(all_comments)
df1

""


In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       148 non-null    object
 1   1       145 non-null    object
 2   2       144 non-null    object
 3   3       144 non-null    object
 4   4       144 non-null    object
 5   5       143 non-null    object
 6   6       143 non-null    object
 7   7       142 non-null    object
 8   8       142 non-null    object
 9   9       142 non-null    object
 10  10      142 non-null    object
 11  11      141 non-null    object
 12  12      141 non-null    object
 13  13      141 non-null    object
 14  14      141 non-null    object
 15  15      141 non-null    object
 16  16      141 non-null    object
 17  17      141 non-null    object
 18  18      141 non-null    object
 19  19      141 non-null    object
 20  20      141 non-null    object
 21  21      140 non-null    object
 22  22      140 non-null    ob

In [12]:
df1.to_csv("tapei_comments_1.csv", encoding = "UTF-8-sig")